In [ ]:
\list

## Exploring 'keywords_all' Column
A brief exploration of the 'keywords_all' column in the 'data_nerd_jobs' table, focusing on the structure and content of the data.

In [ ]:
SELECT keywords_all FROM public_job_listings.data_nerd_jobs;

## Unpacking JSON Objects in 'keywords_all'
Extracting job skills from the JSON objects in the 'keywords_all' column of the 'data_nerd_jobs' table.

In [ ]:
SELECT
  keyword.element
FROM
  public_job_listings.data_nerd_jobs,
  UNNEST(keywords_all.list) AS keyword
LIMIT 100;

## Median Salary for Top 10 Skills
Calculating the median salary for the top 10 skills listed in the 'data_nerd_jobs' table.

In [ ]:
WITH SkillFrequency AS (
  SELECT
    keyword.element AS skill,
    COUNT(*) AS frequency
  FROM
    public_job_listings.data_nerd_jobs,
    UNNEST(keywords_all.list) AS keyword
  GROUP BY skill
  ORDER BY frequency DESC
  LIMIT 10
),
MedianSalaries AS (
  SELECT
    sf.skill,
    PERCENTILE_CONT(salary_year, 0.5) OVER (PARTITION BY sf.skill) AS median_salary
  FROM
    public_job_listings.data_nerd_jobs dj
  JOIN SkillFrequency sf ON dj.keywords_all.list CONTAINS STRUCT(sf.skill)
)
SELECT
  skill,
  median_salary
FROM
  MedianSalaries
GROUP BY skill, median_salary;